In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,105 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [1,079 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.dow

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [5]:
# df with total_votes count equal or greater than 20
votes_20 = df.filter(df.total_votes >= 20)
votes_20.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [7]:
# df with rows where the number of helpful_votes divided by total_votes is >= 50%
helpful_50 = votes_20.filter((votes_20.helpful_votes / votes_20.total_votes) >= 0.5)
helpful_50.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [9]:
# paid vine df
paid_vine_df = helpful_50.filter(helpful_50.vine =='Y')
paid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35353346|R3R2RHQWEKX5NL|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App...|          5|           81|         89|   Y|                N|You can hear your...|First off one bit...| 2014-01-13|
|         US|   50929595|R1RZ4JZO8DAP7H|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App

In [10]:
# unpaid vine df
unpaid_vine_df = helpful_50.filter(helpful_50.vine=='N')
unpaid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [12]:
# count of paid reviews
total_paid = paid_vine_df.count()
total_paid

3

In [17]:
# paid five star reviews
total_paid_five = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
total_paid_five

2

In [13]:
# count of unpaid review
total_unpaid = unpaid_vine_df.count()
total_unpaid

3094

In [18]:
# unpaid five star reviews
total_unpaid_five = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).count()
total_unpaid_five

1704

In [19]:
# paid five star reviews percentage
paid_percent = (total_paid_five / total_paid) * 100
paid_percent

66.66666666666666

In [20]:
# unpaid five star reviews percentage
unpaid_percent = (total_unpaid_five / total_unpaid) * 100
unpaid_percent

55.07433742727861